In [1]:
import os


In [2]:
%pwd


'd:\\A DATA SCIENCE\\personal project\\Deep learning Project\\chicken\\chicken_disease\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'd:\\A DATA SCIENCE\\personal project\\Deep learning Project\\chicken\\chicken_disease'

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [16]:
from dataclasses import dataclass
from pathlib import Path
import os
import time
import tensorflow as tf

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(str(self.config.tensorboard_root_log_dir), f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=str(self.config.checkpoint_model_filepath), save_best_only=True)

    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(str(self.config.updated_base_model_path))

    def train_valid_generator(self):
        datagenerator_kwargs = dict(rescale = 1./255, validation_split=0.20)
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1], batch_size=self.config.params_batch_size, interpolation="bilinear")

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.train_generator = valid_datagenerator.flow_from_directory(directory=str(self.config.training_data), subset="training", shuffle=True, **dataflow_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(directory=str(self.config.training_data), subset="validation", shuffle=True, **dataflow_kwargs)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(str(path))

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)
    
except Exception as e:
    raise e


[2023-11-05 12:11:19,670: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-05 12:11:19,673: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-05 12:11:19,674: INFO: common: created directory at: artifacts]
[2023-11-05 12:11:19,676: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-11-05 12:11:19,677: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-11-05 12:11:19,678: INFO: common: created directory at: artifacts\training]
Found 312 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
19/19 [==============================] - 33s 2s/step - loss: 10.1464 - accuracy: 0.6081 - val_loss: 1.6174 - val_accuracy: 0.7656


c:\Users\HITENDRA\anaconda3\envs\chicken1\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
